## 1. 聚合方法
- 分组后得到的GroupBy对象可以使用聚合方法进行数据聚合，以下是一些常用的聚合方法：
![title](img/数据聚合.png)
- GroupBy对象还可以调用被分组对象里自带的任何方法（非聚合方法也可以）
- 自定义的聚合函数，只需使用`aggregate()`或`agg()`方法传入自定义函数，分组结果的各个切片调用自定义函数并返回运算结果

In [1]:
import numpy as np
import pandas as pd

In [13]:
df = pd.DataFrame({'key1': list('ABBCBCAA'),
                   'key2': list('YZXYZXYZ'),
                   'data1': np.random.randint(100, size=8),
                   'data2': np.random.randint(10, size=8)})
df

,key1,key2,data1,data2
0,A,Y,97,8
1,B,Z,13,0
2,B,X,83,3
3,C,Y,10,0
4,B,Z,80,1
5,C,X,5,5
6,A,Y,46,2
7,A,Z,39,4


In [18]:
# 自定义函数进行聚合操作
def test(arr):
    if len(arr)>1:
        return max(arr)-min(arr)
    else:
        return arr
df.groupby(['key1','key2']).agg(test)

data1  data2
key1 key2              
A    Y        51      6
     Z        39      4
B    X        83      3
     Z        67      1
C    X         5      5
     Y        10      0